In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.3.0).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertConfig
from transformers import BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROTBERT")
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=512,
    hidden_size=128,
    num_hidden_layers=4,
    num_attention_heads=8,
    type_vocab_size=1,
)
model = BertForMaskedLM(config=config)
#model = AutoModelForMaskedLM.from_pretrained("mlewand/PROTBERT-small")

In [7]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.mask_token_id)
print(tokenizer.pad_token_id)

0
2
4
1


# Imports

In [8]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter

import torch

# Hyperparameters

In [26]:
file_id = '1w3IQMK3PmXH-Bq6Lt_P8wxGPvr5olrZT'
destination = 'pfam.zip'
hf_name = "t5-small"
n_families_of_interest = 25
data_dirpath = "pfam"
aminoacid_separate_by = ""
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

# Utils

In [27]:
def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

In [11]:
# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm.tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")

# Get the Dataset

In [12]:
download_file_from_google_drive(file_id, destination)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(".")
!mv random_split pfam

written : 468.781 MB: : 15780it [00:11, 1384.58it/s]




saved all the data to pfam.zip. total size : 493.095 MB
mv: cannot move 'random_split' to 'pfam/random_split': Directory not empty


# Read Data

In [28]:
def read_all_shards(partition='dev', data_dir = data_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    
    return pd.concat(shards)

def read_all_data_initial():
  global train, test, dev, all_train_ds_size, all_test_ds_size, all_dev_ds_size

  test = read_all_shards('test')
  dev = read_all_shards('dev')
  train = read_all_shards('train')

  partitions = {'test': test, 'dev': dev, 'train': train}
  for name, df in partitions.items():
      print('Dataset partition "%s" has %d sequences' % (name, len(df)))

  all_train_ds_size = len(train)
  all_test_ds_size = len(test)
  all_dev_ds_size = len(dev)

  train.reset_index(inplace=True, drop=True)
  dev.reset_index(inplace=True, drop=True)
  test.reset_index(inplace=True, drop=True)

def get_cumulative(data):
    counter = Counter(data['family_accession'])
    print(f"how many labels : {len(counter)}")
    
    datasetSize = len(data)
    xs = []
    x_labels = []
    ys = []

    t = 0
    cumulative = []

    for i,(x, y) in  enumerate(counter.most_common()):
        xs.append(i)
        x_labels.append(x)
        ys.append(y)
        t += y / datasetSize
        cumulative.append(t)
    return cumulative


# EXECUTION CODE
print('Available dataset partitions: ', os.listdir(data_dirpath))
read_all_data_initial()
cumulative = get_cumulative(train)
print(f"{n_families_of_interest} classes is {100 * round( cumulative[n_families_of_interest-1],3)} portion of training data")

familiesOfInterest = train.family_accession.value_counts()[:n_families_of_interest]

mask = train.family_accession.isin(familiesOfInterest.index.values)
train = train.loc[mask,:]

mask = dev.family_accession.isin(familiesOfInterest.index.values)
dev = dev.loc[mask,:]

mask = test.family_accession.isin(familiesOfInterest.index.values)
test = test.loc[mask,:]


################################################################################
train_seq = train['sequence']
dev_seq = dev['sequence']
test_seq = test['sequence']

################################################################################
train_sentences = train_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
validation_sentences = dev_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
test_sentences = test_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))

################################################################################
train_labels = train['family_accession'].apply(lambda x: x.split('.')[0])
validation_labels = dev['family_accession'].apply(lambda x: x.split('.')[0])
test_labels = test['family_accession'].apply(lambda x: x.split('.')[0])

Available dataset partitions:  ['train', 'test', 'random_split', 'dev']
Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
how many labels : 17929
25 classes is 3.1 portion of training data


In [29]:
train_sentences

10                                      SLKFLNFAQNEFNGSIPESV
37         KRVIDIVFSIIGIVVLSPLMLAICIYIKISSKGNAIFSHIRLGYRG...
116        FVLGLVAGSLSFGGAYTAIPFVQVEAVLKGGWLPQSVFIDCIAIGN...
189        KRLIDILGAIVGLIITLLVSVPVAIITLIIDPGPILYSQIRCGLNG...
192                                   SLLQLQLDSTPIEALPEEIGDL
                                 ...                        
1086628    ILDGNATAAAIKEELKERVEALKAKGITPGLGTVLVGDDPASRSYV...
1086658    TGLIQDLGRVERVIPGGMTDLWIHTSLGAAAFELGESIAVNGACLT...
1086682    CETCKGSGGREKAKPHDCSVCGGRGQTLRIQTVGGMLSQVTTQCAN...
1086684    LNAVKLLARVAETRSFTQAAVSLGLTQSGLSRAIGRLEAELGVRLL...
1086720    VLDIGCGNGKITAQIAERVPAGAVLGIDSSEQMISFASSHFAPADW...
Name: sequence, Length: 33322, dtype: object

In [30]:
labels = set()

for v in tqdm.tqdm(train_labels):
  labels.add(v)

label_to_id = {k:v for v,k in enumerate(labels)}

print()
print(len(labels))

100%|██████████| 33322/33322 [00:00<00:00, 1656583.04it/s]


25


In [31]:
from datasets import Dataset
dataset = Dataset.from_dict({"sentence": train_sentences})
dataset_val = Dataset.from_dict({"sentence": validation_sentences})

In [32]:
print(dataset)
print(dataset_val)

Dataset({
    features: ['sentence'],
    num_rows: 33322
})
Dataset({
    features: ['sentence'],
    num_rows: 4151
})


In [33]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint)
print(f"datapoint : {datapoint}, len({len(datapoint)})")
print(encoded)

datapoint : SLKFLNFAQNEFNGSIPESV, len(20)
[0, 55, 48, 47, 42, 48, 50, 42, 37, 53, 50, 41, 42, 50, 43, 55, 45, 52, 41, 55, 58, 2]


In [34]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [35]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/33322 [00:00<?, ? examples/s]

Map:   0%|          | 0/4151 [00:00<?, ? examples/s]

In [36]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.05)
tok = tokenizer(train_sentences.iloc[0], truncation=True)
data_collator([tok])

{'input_ids': tensor([[  0,  55,  48,  47,  42,  48,  50,  42, 102,  53,  50,  41,  42,  50,
          43,  55,  45,  52,   4,  55,  58,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, -100, -100,   37, -100, -100, -100,
         -100, -100, -100, -100, -100, -100,   41, -100, -100, -100]])}

In [22]:
def compute_metrics(pred):
  labels = pred.label_ids
  # labels = np.argmax(labels, axis=1)
  preds = pred.predictions

  tot = np.sum(labels != -100) + 1e-10
  classified_correctly = 0
  for i in range(preds.shape[0]):
    mask = preds[i, :] != -100
    classified_correctly += np.sum(preds[i, mask] == labels[i, mask])
  
  return {"exact_matches": classified_correctly / tot}

In [23]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids

In [40]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True,
    push_to_hub=True,
    hub_model_id="mlewand/PROTBERT-small"
    # eval_accumulation_steps = 8 # Transfer GPU tensors during validation to RAM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    preprocess_logits_for_metrics= preprocess_logits_for_metrics
)

trainer.train()

/workspace/PROTNAME is already a clone of https://huggingface.co/mlewand/PROTBERT-small. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Exact Matches
1,1.576100,1.660719,0.497469
2,1.571500,1.692904,0.492135
3,1.582800,1.682675,0.493190
4,1.631400,1.678565,0.491108
5,1.649200,1.663419,0.494854
6,1.642300,1.664066,0.493056
7,1.642800,1.652631,0.496480
8,1.642400,1.656419,0.498571
9,1.639500,1.665413,0.494800
10,1.641000,1.676204,0.495479


TrainOutput(global_step=2610, training_loss=1.6219058917400024, metrics={'train_runtime': 371.1778, 'train_samples_per_second': 897.737, 'train_steps_per_second': 7.032, 'total_flos': 2309113536295104.0, 'train_loss': 1.6219058917400024, 'epoch': 10.0})

In [ ]:
from  transformers import pipeline

model=model.to("cpu")
p = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
seq = train_sentences.iloc[0]
seq = seq[:-1]
seq += '<mask>'

p(seq)

In [ ]:
!pip install torchinfo
from torchinfo import summary 

In [ ]:
summary(model)